In [1]:
import numpy as np 
import pandas as pd 

In [7]:
df = pd.read_csv(r"C:\Users\Sejal Hanmante\Downloads\selected_crops_temp.csv",index_col='SR NO.')
df.head()

,CROP TYPE,SOIL TYPE,REGION,WEATHER,TEMP_MIN,TEMP.MAX,WATER REQUIREMENT
SR NO.,,,,,,,
0,POTATO,DRY,DESERT,NORMAL,10,20,8.5
1,POTATO,DRY,DESERT,SUNNY,10,20,10.0
2,POTATO,DRY,DESERT,WINDY,10,20,9.5
3,POTATO,DRY,DESERT,RAINY,10,20,0.5
4,POTATO,DRY,DESERT,NORMAL,20,30,9.5


In [9]:
# Specifically choosing Potato as the crop type 
df = df.loc[df['CROP TYPE']=='POTATO']
df.head()

,CROP TYPE,SOIL TYPE,REGION,WEATHER,TEMP_MIN,TEMP.MAX,WATER REQUIREMENT
SR NO.,,,,,,,
0,POTATO,DRY,DESERT,NORMAL,10,20,8.5
1,POTATO,DRY,DESERT,SUNNY,10,20,10.0
2,POTATO,DRY,DESERT,WINDY,10,20,9.5
3,POTATO,DRY,DESERT,RAINY,10,20,0.5
4,POTATO,DRY,DESERT,NORMAL,20,30,9.5


In [11]:
df = df.drop(['CROP TYPE'],axis=1)

In [15]:
# One Hot Encoding

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Define the columns for one-hot encoding
columns_to_encode = ['SOIL TYPE', 'REGION', 'WEATHER']

# Define the ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(sparse_output=False), columns_to_encode)
    ],
    remainder='passthrough'  # Keep the rest of the columns
)

# Apply the transformation
transformed_data = column_transformer.fit_transform(df)

# Get feature names for one-hot encoded columns
encoded_feature_names = column_transformer.named_transformers_['onehot'].get_feature_names_out(columns_to_encode)

# Combine feature names (one-hot encoded + passthrough)
passthrough_columns = [col for col in df.columns if col not in columns_to_encode]
all_feature_names = list(encoded_feature_names) + passthrough_columns

# Create a new DataFrame with the transformed data and feature names
df_transformed = pd.DataFrame(transformed_data, columns=all_feature_names)

# Display the transformed DataFrame
df_transformed


,SOIL TYPE_DRY,SOIL TYPE_HUMID,SOIL TYPE_WET,REGION_DESERT,REGION_HUMID,REGION_SEMI ARID,REGION_SEMI HUMID,WEATHER_NORMAL,WEATHER_RAINY,WEATHER_SUNNY,WEATHER_WINDY,TEMP_MIN,TEMP.MAX,WATER REQUIREMENT
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,20.0,8.50
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10.0,20.0,10.00
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,20.0,9.50
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0,20.0,0.50
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,30.0,9.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,30.0,40.0,0.20
188,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,40.0,50.0,5.15
189,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,40.0,50.0,6.65
190,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,50.0,6.15


In [17]:
X = df_transformed.drop(['WATER REQUIREMENT'],axis=1)
y = df_transformed['WATER REQUIREMENT']

## Random Forest

In [21]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [57]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=300,n_jobs=-1,)
model.fit(X_train,y_train)

rf_train_score = model.score(X_train,y_train)*100 
rf_test_score = model.score(X_test, y_test)*100

## Gradient Boosting

In [28]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor 
reg = GradientBoostingRegressor(random_state=0,n_estimators=250,learning_rate=1.8)
reg.fit(X_train,y_train)
reg.score(X_train,y_train)*100 , reg.score(X_test,y_test)*100

(99.98419256425292, 94.43621309925334)

## XGBoost

In [32]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

xgb_reg = XGBRegressor(random_state=0, n_estimators=250, learning_rate=1.8, verbosity=0)
xgb_reg.fit(X_train, y_train)

xgb_train_score = xgb_reg.score(X_train, y_train) * 100
xgb_test_score = xgb_reg.score(X_test, y_test) * 100

print("XGBoost: ", xgb_train_score, xgb_test_score)


XGBoost:  99.99999403044136 93.7364497406791


## LightBoost

In [35]:
from lightgbm import LGBMRegressor

lgbm_reg = LGBMRegressor(random_state=0, n_estimators=250, learning_rate=1.8)
lgbm_reg.fit(X_train, y_train)

lgbm_train_score = lgbm_reg.score(X_train, y_train) * 100
lgbm_test_score = lgbm_reg.score(X_test, y_test) * 100

print("LightGBM: ", lgbm_train_score, lgbm_test_score)


D:\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001189 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 32
[LightGBM] [Info] Number of data points in the train set: 134, number of used features: 13
[LightGBM] [Info] Start training from score 5.884254
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

## CatBoost

In [40]:
from catboost import CatBoostRegressor

cat_reg = CatBoostRegressor(random_state=0, iterations=250, learning_rate=0.8, verbose=0)
cat_reg.fit(X_train, y_train)

cat_train_score = cat_reg.score(X_train, y_train) * 100
cat_test_score = cat_reg.score(X_test, y_test) * 100

print("CatBoost: ", cat_train_score, cat_test_score)


CatBoost:  99.99999999999292 97.81478960249781


## MLPRegressor

In [42]:
from sklearn.neural_network import MLPRegressor

nn_reg = MLPRegressor(random_state=0, hidden_layer_sizes=(128, 64), max_iter=500, learning_rate_init=0.01)
nn_reg.fit(X_train, y_train)

nn_train_score = nn_reg.score(X_train, y_train) * 100
nn_test_score = nn_reg.score(X_test, y_test) * 100

print("Neural Network: ", nn_train_score, nn_test_score)


Neural Network:  96.86374150893784 96.91883040398429


## SVR

In [46]:
from sklearn.svm import SVR

svr_reg = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_reg.fit(X_train, y_train)

svr_train_score = svr_reg.score(X_train, y_train) * 100
svr_test_score = svr_reg.score(X_test, y_test) * 100

print("SVR: ", svr_train_score, svr_test_score)


SVR:  0.6969259760640667 8.39273690542095


In [63]:
results = {
    "Random Forest" : (rf_train_score ,rf_test_score ),
    "GradientBoosting": (reg.score(X_train, y_train) * 100, reg.score(X_test, y_test) * 100),
    "XGBoost": (xgb_train_score, xgb_test_score),
    "LightGBM": (lgbm_train_score, lgbm_test_score),
    "CatBoost": (cat_train_score, cat_test_score),
    "NeuralNetwork": (nn_train_score, nn_test_score),
}

for model, scores in results.items():
    print(f"{model}: Train Score: {scores[0]:.2f}%, Test Score: {scores[1]:.2f}%")


Random Forest: Train Score: 99.66%, Test Score: 97.91%
GradientBoosting: Train Score: 99.98%, Test Score: 94.44%
XGBoost: Train Score: 100.00%, Test Score: 93.74%
LightGBM: Train Score: 99.37%, Test Score: 96.59%
CatBoost: Train Score: 100.00%, Test Score: 97.81%
NeuralNetwork: Train Score: 96.86%, Test Score: 96.92%


In [65]:
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Train Score (%)', 'Test Score (%)'])
results_df

,Train Score (%),Test Score (%)
Random Forest,99.662800,97.905765
GradientBoosting,99.984193,94.436213
XGBoost,99.999994,93.736450
LightGBM,99.367424,96.588048
CatBoost,100.000000,97.814790
NeuralNetwork,96.863742,96.918830
